# P7: Disaster Relief Deployment - Multiagent Pattern


<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together.

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.

**P7: Disaster Relief Deployment** - Planning and scheduling to allocate aid and personnel across regions under uncertainty and evolving needs.


In [ ]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


In [ ]:
import sys
print("Python version:", sys.version)


In [ ]:
%pip install openai python-dotenv graphviz colorama


In [ ]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")


In [ ]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)


## The Agent Class


First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.


In [ ]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")


You can also associate tools with the agent. Let's create a tool for writing some string into a file.


In [ ]:
from src.tool_agent.tool import tool

@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")


## The Crew for P7: Disaster Relief Deployment


In [ ]:
from src.multi_agent.crew import Crew

with Crew() as crew:
    # ---- Situation Assessment Agent ---- #
    SA_Agent = Agent(
        name="Situation Assessment Agent",
        backstory="You assess disaster situations, identify affected regions, severity levels, and resource needs.",
        task_description="Evaluate disaster impact: identify regions (region1: critical, needs medical/food; region2: urgent, needs water/shelter), assess severity, and determine resource requirements.",
        task_expected_output="Structured assessment: 1) Affected regions with severity levels. 2) Resource needs per region (medical supplies, food, water, shelter). 3) Emergency response deadlines (critical: 2h, urgent: 6h, normal: 24h)."
    )
    
    # ---- Resource Inventory Agent ---- #
    RI_Agent = Agent(
        name="Resource Inventory Agent",
        backstory="You manage available relief resources including supplies, personnel, and vehicles.",
        task_description="Inventory available resources: medical supplies (1000 units), food (5000 units), water (2000 units), personnel (20 doctors, 50 nurses, 30 logistics), and vehicles (helicopters, trucks).",
        task_expected_output="Complete resource inventory with quantities, locations, and capabilities of all available relief resources."
    )
    
    # ---- Allocation Planning Agent ---- #
    AP_Agent = Agent(
        name="Allocation Planning Agent",
        backstory="You plan optimal allocation of resources to affected regions to maximize aid delivery and minimize response time.",
        task_description="Create allocation plan distributing resources to regions based on needs, severity, and deadlines, optimizing for maximum aid delivery and minimum response time.",
        task_expected_output="Resource allocation plan: assignments of supplies, personnel, and vehicles to each region with delivery schedules."
    )
    
    # ---- Personnel Deployment Agent ---- #
    PD_Agent = Agent(
        name="Personnel Deployment Agent",
        backstory="You deploy personnel teams to regions based on their skills and regional needs.",
        task_description="Deploy personnel teams (medical teams, logistics teams) to regions matching skills with needs, ensuring timely arrival within deadlines.",
        task_expected_output="Personnel deployment schedule with team assignments, routes, and arrival times for each region."
    )
    
    # ---- Vehicle Routing Agent ---- #
    VR_Agent = Agent(
        name="Vehicle Routing Agent",
        backstory="You plan vehicle routes for delivering supplies and personnel to affected regions.",
        task_description="Plan routes for vehicles (helicopters for fast delivery, trucks for bulk transport) to deliver resources to regions, considering distance, urgency, and vehicle capabilities.",
        task_expected_output="Vehicle routing plan with delivery schedules, routes, and estimated arrival times for each vehicle."
    )
    
    # ---- Contingency Planning Agent ---- #
    CP_Agent = Agent(
        name="Contingency Planning Agent",
        backstory="You develop contingency plans for weather events, resource shortages, and other disruptions.",
        task_description="Create contingency plans for potential disruptions: weather events blocking transport, resource shortages, and alternative allocation strategies.",
        task_expected_output="Contingency plans with alternative routes, backup resource sources, and adaptive strategies for disruptions."
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './p7_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    SA_Agent >> RI_Agent >> AP_Agent >> PD_Agent >> VR_Agent >> CP_Agent >> Writer_agent


In [ ]:
crew.run()
